# Spotify Business Intelligence & Product Insights

## Objective
Extract actionable business insights from Spotify streaming data to inform product decisions, user engagement strategies, and platform optimization efforts.

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Load cleaned data
df = pd.read_csv('../data/spotify_cleaned.csv')
session_stats = pd.read_csv('../data/session_statistics.csv')

# Convert timestamp back to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(f"Loaded {len(df):,} streaming records")
print(f"Loaded {len(session_stats):,} session records")

## 2. Peak Usage Analysis & Platform Strategy

In [ ]:
# Peak usage times analysis for resource planning
hourly_usage = df.groupby('hour_of_day').agg({
    'spotify_track_uri': 'count',
    'seconds_played': 'sum',
    'is_skip': 'mean'
}).round(3)

hourly_usage.columns = ['Total_Plays', 'Total_Seconds', 'Skip_Rate']
hourly_usage['Total_Hours'] = hourly_usage['Total_Seconds'] / 3600

# Create interactive peak usage visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Hourly Play Volume', 'Hourly Listening Time', 
                   'Platform Usage by Hour', 'Skip Rate by Hour'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Hourly plays
fig.add_trace(
    go.Bar(x=hourly_usage.index, y=hourly_usage['Total_Plays'], 
           name='Total Plays', marker_color='skyblue'),
    row=1, col=1
)

# Hourly listening time
fig.add_trace(
    go.Bar(x=hourly_usage.index, y=hourly_usage['Total_Hours'], 
           name='Total Hours', marker_color='lightcoral'),
    row=1, col=2
)

# Platform usage by hour (stacked bar)
platform_hourly = df.groupby(['hour_of_day', 'platform']).size().unstack(fill_value=0)
for platform in platform_hourly.columns:
    fig.add_trace(
        go.Bar(x=platform_hourly.index, y=platform_hourly[platform], 
               name=platform),
        row=2, col=1
    )

# Skip rate by hour
fig.add_trace(
    go.Scatter(x=hourly_usage.index, y=hourly_usage['Skip_Rate']*100, 
              mode='lines+markers', name='Skip Rate %', 
              line=dict(color='orange', width=3)),
    row=2, col=2
)

fig.update_layout(height=700, title_text="Peak Usage Analysis Dashboard")
fig.show()

# Business insights
peak_hour = hourly_usage['Total_Plays'].idxmax()
low_hour = hourly_usage['Total_Plays'].idxmin()
peak_skip_hour = hourly_usage['Skip_Rate'].idxmax()

print(f"📊 PEAK USAGE INSIGHTS:")
print(f"Peak usage hour: {peak_hour}:00 ({hourly_usage.loc[peak_hour, 'Total_Plays']:,} plays)")
print(f"Lowest usage hour: {low_hour}:00 ({hourly_usage.loc[low_hour, 'Total_Plays']:,} plays)")
print(f"Highest skip rate: {peak_skip_hour}:00 ({hourly_usage.loc[peak_skip_hour, 'Skip_Rate']:.1%})")
print(f"Peak usage is {hourly_usage.loc[peak_hour, 'Total_Plays'] / hourly_usage.loc[low_hour, 'Total_Plays']:.1f}x higher than lowest")

## 3. Platform Performance & User Experience Analysis

In [ ]:
# Comprehensive platform analysis
platform_metrics = df.groupby('platform').agg({
    'spotify_track_uri': 'count',
    'seconds_played': ['sum', 'mean'],
    'is_skip': 'mean',
    'percent_played': 'mean',
    'shuffle': 'mean'
}).round(3)

platform_metrics.columns = ['Total_Plays', 'Total_Seconds', 'Avg_Seconds_Per_Play', 
                            'Skip_Rate', 'Avg_Percent_Played', 'Shuffle_Usage_Rate']

platform_metrics['Total_Hours'] = platform_metrics['Total_Seconds'] / 3600
platform_metrics['Market_Share'] = (platform_metrics['Total_Plays'] / platform_metrics['Total_Plays'].sum() * 100).round(1)

print("🎯 PLATFORM PERFORMANCE DASHBOARD")
print("=" * 50)
print(platform_metrics)

# Platform engagement quality score
# Higher percent played + lower skip rate + longer sessions = better engagement
platform_metrics['Engagement_Score'] = (
    (platform_metrics['Avg_Percent_Played'] / platform_metrics['Avg_Percent_Played'].max()) * 0.4 +
    ((1 - platform_metrics['Skip_Rate']) / (1 - platform_metrics['Skip_Rate']).max()) * 0.4 +
    (platform_metrics['Avg_Seconds_Per_Play'] / platform_metrics['Avg_Seconds_Per_Play'].max()) * 0.2
) * 100

print(f"\n📱 PLATFORM RANKINGS:")
platform_ranking = platform_metrics.sort_values('Engagement_Score', ascending=False)
for i, (platform, score) in enumerate(platform_ranking['Engagement_Score'].items(), 1):
    print(f"{i}. {platform}: {score:.1f} engagement score")

# Platform switching analysis
platform_switches = df.sort_values('timestamp')
platform_switches['prev_platform'] = platform_switches['platform'].shift(1)
platform_switches['platform_switch'] = platform_switches['platform'] != platform_switches['prev_platform']
switch_rate = platform_switches['platform_switch'].mean()

print(f"\n🔄 Platform switch rate: {switch_rate:.1%}")

## 4. Skip Behavior Deep Dive & Content Strategy

In [ ]:
# Advanced skip analysis for content strategy
# Skip drivers analysis
skip_drivers = {
    'Platform': df.groupby('platform')['is_skip'].mean(),
    'Time of Day': df.groupby('time_of_day')['is_skip'].mean(),
    'Shuffle Mode': df.groupby('shuffle')['is_skip'].mean(),
    'Day of Week': df.groupby('day_of_week')['is_skip'].mean()
}

print("🎵 SKIP BEHAVIOR ANALYSIS")
print("=" * 40)
for category, data in skip_drivers.items():
    print(f"\n{category}:")
    for item, rate in data.sort_values(ascending=False).items():
        print(f"  {item}: {rate:.1%}")

# Track length vs skip analysis
df['track_length_category'] = pd.cut(df['estimated_track_length_ms']/1000, 
                                    bins=[0, 120, 180, 240, 300, float('inf')],
                                    labels=['<2min', '2-3min', '3-4min', '4-5min', '5min+'])

length_skip_analysis = df.groupby('track_length_category').agg({
    'is_skip': ['mean', 'count'],
    'percent_played': 'mean'
}).round(3)

print(f"\n🎵 TRACK LENGTH vs SKIP BEHAVIOR:")
print(length_skip_analysis)

# Create skip prediction features importance
skip_features = df[['hour_of_day', 'day_of_week_num', 'estimated_track_length_ms', 
                   'shuffle', 'platform']].copy()
skip_features = pd.get_dummies(skip_features, columns=['platform'])
skip_features['shuffle'] = skip_features['shuffle'].astype(int)

# Correlation with skip behavior
correlations = skip_features.corrwith(df['is_skip'].astype(int)).sort_values(key=abs, ascending=False)
print(f"\n🔍 SKIP PREDICTION FEATURE IMPORTANCE:")
for feature, corr in correlations.items():
    if abs(corr) > 0.01:  # Only show meaningful correlations
        print(f"  {feature}: {corr:.3f}")

## 5. User Engagement & Retention Analysis

In [ ]:
# Daily and weekly activity analysis (retention proxy)
df['date'] = df['timestamp'].dt.date
daily_activity = df.groupby('date').agg({
    'spotify_track_uri': 'count',
    'seconds_played': 'sum',
    'is_skip': 'mean'
}).round(3)

daily_activity.columns = ['Daily_Plays', 'Daily_Seconds', 'Daily_Skip_Rate']
daily_activity['Daily_Hours'] = daily_activity['Daily_Seconds'] / 3600

# Weekly patterns
df['week'] = df['timestamp'].dt.isocalendar().week
weekly_activity = df.groupby('week').agg({
    'spotify_track_uri': 'count',
    'seconds_played': 'sum',
    'date': 'nunique'  # Active days per week
}).round(3)

weekly_activity.columns = ['Weekly_Plays', 'Weekly_Seconds', 'Active_Days']
weekly_activity['Weekly_Hours'] = weekly_activity['Weekly_Seconds'] / 3600

print("📈 USER ACTIVITY & RETENTION METRICS")
print("=" * 45)
print(f"Average daily plays: {daily_activity['Daily_Plays'].mean():.0f}")
print(f"Average daily listening: {daily_activity['Daily_Hours'].mean():.1f} hours")
print(f"Most active day: {daily_activity['Daily_Plays'].max():,.0f} plays")
print(f"Least active day: {daily_activity['Daily_Plays'].min():,.0f} plays")
print(f"Activity consistency (CV): {(daily_activity['Daily_Plays'].std() / daily_activity['Daily_Plays'].mean()):.2f}")

# Engagement trends over time
daily_activity.reset_index(inplace=True)
daily_activity['date'] = pd.to_datetime(daily_activity['date'])
daily_activity = daily_activity.sort_values('date')

# Calculate 7-day rolling averages
daily_activity['Rolling_Plays'] = daily_activity['Daily_Plays'].rolling(7, center=True).mean()
daily_activity['Rolling_Hours'] = daily_activity['Daily_Hours'].rolling(7, center=True).mean()

# Visualize trends
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Daily plays trend
ax1.plot(daily_activity['date'], daily_activity['Daily_Plays'], alpha=0.3, label='Daily')
ax1.plot(daily_activity['date'], daily_activity['Rolling_Plays'], color='red', linewidth=2, label='7-day Average')
ax1.set_title('Daily Listening Activity Trend')
ax1.set_ylabel('Number of Plays')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Daily hours trend
ax2.plot(daily_activity['date'], daily_activity['Daily_Hours'], alpha=0.3, label='Daily')
ax2.plot(daily_activity['date'], daily_activity['Rolling_Hours'], color='blue', linewidth=2, label='7-day Average')
ax2.set_title('Daily Listening Time Trend')
ax2.set_ylabel('Hours')
ax2.set_xlabel('Date')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Session Quality & User Journey Analysis

In [ ]:
# Session quality scoring
session_stats['Session_Quality_Score'] = (
    (session_stats['Session_Duration_Minutes'] / session_stats['Session_Duration_Minutes'].max()) * 0.3 +
    (session_stats['Tracks_Count'] / session_stats['Tracks_Count'].max()) * 0.3 +
    ((1 - session_stats['Skip_Rate']) / (1 - session_stats['Skip_Rate']).max()) * 0.4
) * 100

# Session quality categories
session_stats['Quality_Category'] = pd.cut(session_stats['Session_Quality_Score'],
                                          bins=[0, 33, 66, 100],
                                          labels=['Low', 'Medium', 'High'])

quality_distribution = session_stats['Quality_Category'].value_counts(normalize=True) * 100

print("🏆 SESSION QUALITY ANALYSIS")
print("=" * 35)
print(f"High Quality Sessions: {quality_distribution.get('High', 0):.1f}%")
print(f"Medium Quality Sessions: {quality_distribution.get('Medium', 0):.1f}%")
print(f"Low Quality Sessions: {quality_distribution.get('Low', 0):.1f}%")

# Platform quality comparison
platform_quality = session_stats.groupby('Primary_Platform')['Session_Quality_Score'].mean().sort_values(ascending=False)
print(f"\n📱 PLATFORM SESSION QUALITY:")
for platform, score in platform_quality.items():
    print(f"  {platform}: {score:.1f} avg quality score")

# User journey patterns
# Analyze reason_start and reason_end patterns
journey_analysis = df.groupby(['reason_start', 'reason_end']).agg({
    'spotify_track_uri': 'count',
    'is_skip': 'mean',
    'percent_played': 'mean'
}).round(3)

journey_analysis.columns = ['Count', 'Skip_Rate', 'Avg_Percent_Played']
journey_analysis = journey_analysis[journey_analysis['Count'] >= 100]  # Filter for common patterns
journey_analysis = journey_analysis.sort_values('Count', ascending=False)

print(f"\n🛤️  TOP USER JOURNEY PATTERNS:")
print(journey_analysis.head(10))

## 7. Content Performance & Recommendation Insights

In [ ]:
# Artist and track performance analysis
artist_performance = df.groupby('artist_name').agg({
    'spotify_track_uri': 'count',
    'track_name': 'nunique',
    'seconds_played': 'sum',
    'is_skip': 'mean',
    'percent_played': 'mean',
    'shuffle': 'mean'
}).round(3)

artist_performance.columns = ['Total_Plays', 'Unique_Tracks', 'Total_Seconds', 
                             'Skip_Rate', 'Avg_Percent_Played', 'Shuffle_Rate']

# Filter for artists with significant play count
artist_performance = artist_performance[artist_performance['Total_Plays'] >= 20]
artist_performance['Total_Hours'] = artist_performance['Total_Seconds'] / 3600

# Artist engagement score
artist_performance['Engagement_Score'] = (
    (artist_performance['Avg_Percent_Played'] / artist_performance['Avg_Percent_Played'].max()) * 0.4 +
    ((1 - artist_performance['Skip_Rate']) / (1 - artist_performance['Skip_Rate']).max()) * 0.4 +
    (artist_performance['Total_Plays'] / artist_performance['Total_Plays'].max()) * 0.2
) * 100

# Top performers
top_engaging_artists = artist_performance.sort_values('Engagement_Score', ascending=False).head(10)
most_played_artists = artist_performance.sort_values('Total_Plays', ascending=False).head(10)
least_skipped_artists = artist_performance.sort_values('Skip_Rate').head(10)

print("🎤 ARTIST PERFORMANCE INSIGHTS")
print("=" * 40)
print("\nTop 10 Most Engaging Artists:")
print(top_engaging_artists[['Total_Plays', 'Skip_Rate', 'Engagement_Score']].head())

print("\nTop 10 Least Skipped Artists:")
print(least_skipped_artists[['Total_Plays', 'Skip_Rate', 'Avg_Percent_Played']].head())

# Genre insights (using reason_start as proxy for discovery vs intentional)
discovery_analysis = df.groupby('reason_start').agg({
    'spotify_track_uri': 'count',
    'is_skip': 'mean',
    'percent_played': 'mean'
}).round(3)

discovery_analysis.columns = ['Count', 'Skip_Rate', 'Avg_Percent_Played']
discovery_analysis = discovery_analysis.sort_values('Count', ascending=False)

print(f"\n🔍 CONTENT DISCOVERY PATTERNS:")
print(discovery_analysis)

## 8. Business Recommendations Dashboard

In [ ]:
# Generate actionable business recommendations
print("🎯 BUSINESS RECOMMENDATIONS DASHBOARD")
print("=" * 50)

# 1. Platform Optimization
best_platform = platform_metrics['Engagement_Score'].idxmax()
worst_platform = platform_metrics['Engagement_Score'].idxmin()
print(f"\n📱 PLATFORM STRATEGY:")
print(f"✅ Invest in {best_platform} (highest engagement: {platform_metrics.loc[best_platform, 'Engagement_Score']:.1f})")
print(f"⚠️  Optimize {worst_platform} user experience (lowest engagement: {platform_metrics.loc[worst_platform, 'Engagement_Score']:.1f})")

# 2. Content Strategy
high_skip_threshold = df['is_skip'].quantile(0.75)
content_recommendations = []

if df[df['shuffle'] == True]['is_skip'].mean() > df[df['shuffle'] == False]['is_skip'].mean():
    content_recommendations.append("Improve shuffle algorithm - higher skip rates in shuffle mode")
    
peak_skip_time = df.groupby('time_of_day')['is_skip'].mean().idxmax()
content_recommendations.append(f"Focus on {peak_skip_time.lower()} playlists - highest skip period")

print(f"\n🎵 CONTENT STRATEGY:")
for i, rec in enumerate(content_recommendations, 1):
    print(f"{i}. {rec}")

# 3. User Engagement
avg_session_quality = session_stats['Session_Quality_Score'].mean()
low_quality_sessions_pct = (session_stats['Session_Quality_Score'] < 33).mean() * 100

print(f"\n👥 USER ENGAGEMENT:")
print(f"• Average session quality: {avg_session_quality:.1f}/100")
print(f"• {low_quality_sessions_pct:.1f}% of sessions are low quality")
print(f"• Target: Reduce low-quality sessions by improving onboarding")

# 4. Peak Usage Optimization
peak_usage_ratio = hourly_usage['Total_Plays'].max() / hourly_usage['Total_Plays'].min()
print(f"\n⏰ INFRASTRUCTURE:")
print(f"• Peak usage is {peak_usage_ratio:.1f}x higher than low periods")
print(f"• Scale infrastructure for {peak_hour}:00 peak demand")
print(f"• Consider off-peak promotions during {low_hour}:00-{(low_hour+2)%24}:00")

# 5. Key Performance Targets
current_skip_rate = df['is_skip'].mean()
target_skip_rate = current_skip_rate * 0.9  # 10% improvement
current_avg_session = session_stats['Session_Duration_Minutes'].mean()
target_session_length = current_avg_session * 1.15  # 15% improvement

print(f"\n🎯 PERFORMANCE TARGETS:")
print(f"• Reduce skip rate from {current_skip_rate:.1%} to {target_skip_rate:.1%}")
print(f"• Increase avg session from {current_avg_session:.1f} to {target_session_length:.1f} minutes")
print(f"• Improve {worst_platform} engagement by 20%")
print(f"• Increase high-quality sessions from {quality_distribution.get('High', 0):.1f}% to 40%")

## 9. Executive Summary Metrics

In [ ]:
# Create executive summary with key insights
total_users_proxy = len(df)  # Using total plays as user proxy
total_content_hours = df['seconds_played'].sum() / 3600
revenue_proxy = total_content_hours * 0.004  # Rough estimate: $0.004 per stream hour

executive_summary = {
    'Total Streaming Events': f"{len(df):,}",
    'Total Content Hours': f"{total_content_hours:,.0f}",
    'Revenue Proxy ($)': f"${revenue_proxy:,.0f}",
    'Platform Market Share': f"Web: {platform_metrics.loc['web player', 'Market_Share']:.0f}%",
    'Average Session Length': f"{session_stats['Session_Duration_Minutes'].mean():.1f} min",
    'Overall Skip Rate': f"{df['is_skip'].mean():.1%}",
    'Peak Usage Hour': f"{peak_hour}:00",
    'Top Platform (Engagement)': f"{best_platform}",
    'High Quality Sessions': f"{quality_distribution.get('High', 0):.1f}%",
    'Content Catalog': f"{df['spotify_track_uri'].nunique():,} tracks"
}

print("📊 EXECUTIVE SUMMARY")
print("=" * 30)
for metric, value in executive_summary.items():
    print(f"{metric:<25}: {value}")

# Save business insights
# Create summary dataframes for export
business_metrics = pd.DataFrame({
    'Metric': list(executive_summary.keys()),
    'Value': list(executive_summary.values())
})

business_metrics.to_csv('../reports/executive_summary.csv', index=False)
platform_metrics.to_csv('../reports/platform_analysis.csv')
artist_performance.to_csv('../reports/artist_performance.csv')

print("\n✅ Business intelligence reports saved to ../reports/")